In [1]:
import pandas as pd
import json
import pycountry
import plotly.express as px

raw_df = pd.read_csv('../../data/yt_raw/yt_w_sentiment.csv')
raw_df.head()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/3642524485.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv('../../data/yt_raw/yt_w_sentiment.csv')


,id,text,username,date,country,likes,n_children,title,platform,meta,lang,sentiment
0,Ugx4UuB8c0ewgGUueC14AaABAg,We just need Korean GPT to write scripts for y...,Lanzer,2023-03-28T08:28:43Z,NaN,4.0,1.0,GPT4 출시! 만능 AI와 글/사진/영상으로 대화하기. 뜨거운 AI 경쟁 비하인드...,youtube,"{'vid_id': 'fIWpPnMW_0A', 'user_id': 'UCygv9lT...",en,"[{""label"": ""NEGATIVE"", ""score"": 0.997246265411..."
1,UgzZhETXTVr5Uu-NqCp4AaABAg,"Ah yes, the end is here",Ache of Rectum,2023-03-28T18:55:54Z,NaN,0.0,0.0,Nvidia Using AI To Make AI,youtube,"{'vid_id': 'lf0_K7VWAng', 'user_id': 'UCO9OhNs...",en,"[{""label"": ""POSITIVE"", ""score"": 0.994859695434..."
2,UgxKHma2lTUBof2Owbt4AaABAg,Ummm is that not how the singularity hits?,Jeremiah Deguzman,2023-03-28T18:21:15Z,NaN,0.0,0.0,Nvidia Using AI To Make AI,youtube,"{'vid_id': 'lf0_K7VWAng', 'user_id': 'UCZQ29dk...",en,"[{""label"": ""NEGATIVE"", ""score"": 0.999614477157..."
3,UgxdpdQ7YGj1CUkNSOl4AaABAg,Well in Only Fans some are using their cuLitho...,Chemy Torres,2023-03-28T18:19:05Z,NaN,0.0,0.0,Nvidia Using AI To Make AI,youtube,"{'vid_id': 'lf0_K7VWAng', 'user_id': 'UC2oYapl...",en,"[{""label"": ""NEGATIVE"", ""score"": 0.966815650463..."
4,UgztIUfPfvCNGm8vyY94AaABAg,😂 cuLitho in Spanish sounds like little-ass 😂,dljimenezv,2023-03-28T17:21:02Z,NaN,1.0,0.0,Nvidia Using AI To Make AI,youtube,"{'vid_id': 'lf0_K7VWAng', 'user_id': 'UCa04vcj...",en,"[{""label"": ""NEGATIVE"", ""score"": 0.997109711170..."


In [2]:
df = raw_df[raw_df['country'].notna()]
df.shape

(4584, 12)

In [3]:
# only use the first model for now
for i, row in df.iterrows():
    label = json.loads(row['sentiment'])[0]['label']
    df.at[i, 'sentiment_0'] = 1 if label == 'POSITIVE' else -1


/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/2937561172.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[i, 'sentiment_0'] = 1 if label == 'POSITIVE' else -1


In [4]:
# get average for each country
country_df = df.groupby('country').mean().reset_index()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/469784714.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_df = df.groupby('country').mean().reset_index()


In [5]:
country_2_to_3 = {country.alpha_2: country.alpha_3 for country in pycountry.countries}
for i, row in country_df.iterrows():
    country = row['country']
    if country in country_2_to_3:
        country_df.at[i, 'country'] = country_2_to_3[country]

In [6]:
fig = px.choropleth(country_df, locations="country", color="sentiment_0",
                    hover_name="country",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.update_layout(
    title_text='Average Sentiment of YouTube Videos by Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)
fig.show()

In [7]:
# now plot the sentiment in each country for each year

# exclude rows with no date
df = df[df['date'].notna()]
df = df[df['date'] != 'CZ'] # invalid value

# get the year from the date
for i, row in df.iterrows():
    date = row['date']
    df.at[i, 'year'] = date.split('-')[0]

# get average for each country
df = df.groupby(['country', 'year']).mean().reset_index()

# convert country to 3 letter code
for i, row in df.iterrows():
    country = row['country']
    if country in country_2_to_3:
        df.at[i, 'country'] = country_2_to_3[country]

# sort by year
df = df.sort_values(by=['year'])

# plot
fig = px.choropleth(df, locations="country", color="sentiment_0",
                    hover_name="country", animation_frame="year",
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.update_layout(
    title_text = 'Sentiment of YouTube Videos by Country',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)
fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/4284186158.py:13: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [8]:
# show line chart averaged over all countries
avg_sentiment = raw_df[raw_df['date'].notna()]
avg_sentiment = avg_sentiment[avg_sentiment['date'] != 'CZ'] # invalid value

# get the month from the date
for i, row in avg_sentiment.iterrows():
    date = row['date']
    avg_sentiment.at[i, 'year-month'] = '-'.join(date.split('-')[:2])

    # only use the first model for now
    label = json.loads(row['sentiment'])[0]['label']
    avg_sentiment.at[i, 'sentiment_0'] = 1 if label == 'POSITIVE' else -1

# get average for each country
avg_sentiment = avg_sentiment.groupby(['year-month']).mean().reset_index()

# sort by year
avg_sentiment = avg_sentiment.sort_values(by=['year-month'])

# plot
fig = px.line(avg_sentiment, x="year-month", y="sentiment_0")
fig.update_layout(
    title="Average Sentiment Over Time",
    xaxis_title="Year-Month",
    yaxis_title="Sentiment",
    legend_title="Sentiment",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/244884552.py:15: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [9]:
# line chart of the number of datapoints over time (per month)

# get the month from the date
df = raw_df[raw_df['date'].notna()]
for i, row in df.iterrows():
    date = row['date']
    df.at[i, 'year-month'] = '-'.join(date.split('-')[:2])

# get average for each country
df = df.groupby(['year-month']).count().reset_index()

# sort by year
df = df.sort_values(by=['year-month'])

# plot
fig = px.line(df, x="year-month", y="text")
fig.update_layout(title_text='Number of datapoints over time')
fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/220791727.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
# get the month from the date
df = raw_df[raw_df['date'].notna()]
for i, row in df.iterrows():
    date = row['date']
    df.at[i, 'year-month'] = '-'.join(date.split('-')[:2])

df[df['year-month'] == '2021-08'].groupby('title').count().sort_values('id', ascending=False).head()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/3188336453.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,text,username,date,country,likes,n_children,platform,meta,lang,sentiment,year-month
title,,,,,,,,,,,,
Tesla Bot: Elon Musk’s AI APOCALYPSE?!!,3192,3192,3192,3192,328,3192,3192,3192,3192,3192,3192,3192
Tesla AI Day Highlights | Lex Fridman,658,658,658,658,107,658,658,658,658,658,658,658
Tesla AI Day in 19 Minutes (SUPERCUT),331,331,331,331,48,331,331,331,331,331,331,331
How AI is driving a future of autonomous warfare | DW Analysis,60,60,60,60,6,60,60,60,60,60,60,60
Fighter Jet Pilot AI Analysis (GTA V),46,46,46,46,13,46,46,46,46,46,46,46


In [13]:
# sentiment over time in the US vs the rest of the world
df = raw_df[raw_df['date'].notna()]

for i, row in df.iterrows():
    # get the month from the date
    date = row['date']
    df.at[i, 'year-month'] = '-'.join(date.split('-')[:2])

    # get the average sentiment
    sentiment = []
    sentiment_json = json.loads(row['sentiment'])
    # distilbert
    score = sentiment_json[0]['score']
    if sentiment_json[0]['label'] == 'NEGATIVE': score *= -1
    sentiment.append(score)
    # roberta
    score = sentiment_json[1]['score']
    if sentiment_json[1]['label'] == 'LABEL_0': score = -1 * score
    elif sentiment_json[1]['label'] == 'LABEL_1': score = 0
    sentiment.append(score)
    # bertweet
    score = sentiment_json[2]['score']
    if sentiment_json[2]['label'] == 'NEG': score = -1 * score
    elif sentiment_json[2]['label'] == 'NEU': score = 0
    sentiment.append(score)
    # average
    df.at[i, 'sentiment_avg'] = sum(sentiment) / len(sentiment)

    # whether the video is from the US or not
    country = row['country']
    df.at[i, 'is_us'] = 'US' if country == 'US' else 'Not US'

# get average for each country
df = df.groupby(['year-month', 'is_us']).mean().reset_index()

# sort by year
df = df.sort_values(by=['year-month'])

# plot
fig = px.line(df, x="year-month", y="sentiment_avg", color='is_us')
fig.update_layout(title_text='Sentiment Over Time in the US vs the Rest of the World')
fig.show()

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/1369517358.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/1369517358.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/4c/w2fyysz50_j_234jdvb_c9wr0000gn/T/ipykernel_52053/1369517358.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [17]:
# plot volume of comments in the US vs rest of the world
df = raw_df[raw_df['date'].notna()]
df = df[df['country'].notna()]

for i, row in df.iterrows():
    # get the month from the date
    date = row['date']
    df.at[i, 'year-month'] = '-'.join(date.split('-')[:2])

    # whether the video is from the US or not
    country = row['country']
    df.at[i, 'is_us'] = 'US' if country == 'US' else 'Not US'

# get average for each country
df = df.groupby(['year-month', 'is_us']).count().reset_index()

# sort by year
df = df.sort_values(by=['year-month'])

# plot
fig = px.line(df, x="year-month", y="text", color='is_us')
fig.update_layout(title_text='Volume of Comments in the US vs the Rest of the World')
fig.show()